In [1]:
import pandas as pd

In [2]:
df_TA=pd.read_excel('TrafficAccident112.xlsx')
df_RW=pd.read_excel('RoadWork112.xlsx')
print(df_TA)
print(df_RW)

          年  月   日   時   分  國道名稱 方向    里程      事件發生  交控中心\n接獲通報  ...  車輛4  \
0      2023  2   9  19  21  國道3號  南  54.0  19:21:00         NaN  ...  NaN   
1      2023  2  10   3  36  國道1號  南   6.0  03:36:00         NaN  ...  NaN   
2      2023  2  10   5  33  國道3號  南  76.0  05:33:00         NaN  ...  NaN   
3      2023  2  10   7  35  國道3號  北  98.0  07:35:00         NaN  ...  NaN   
4      2023  2  10   7  52  國道1號  北  23.0  07:52:00         NaN  ...  NaN   
...     ... ..  ..  ..  ..   ... ..   ...       ...         ...  ...  ...   
26734  2023  8   8   9  35  國道1號  南  23.0  09:35:00         NaN  ...  NaN   
26735  2023  8   8   9  44  國道3號  南  34.0  09:44:00         NaN  ...  NaN   
26736  2023  8   8   9  43  國道1號  北   6.0  09:43:00         NaN  ...  NaN   
26737  2023  8   8   9  42  國道1號  南  39.4  09:42:00         NaN  ...  NaN   
26738  2023  8   8   9  51  國道1號  北  23.3  09:51:00         NaN  ...  NaN   

       車輛5  車輛6  車輛7  車輛8  車輛9 車輛10  車輛11 車輛12  分局  
0      NaN  NaN  NaN  

In [3]:
print(df_RW['incStepBlockagePattern'])
df_RW=df_RW.drop(0)
print(df_RW['incStepBlockagePattern'])

0                         占用車道
1         00000000010000000000
2         00000000010000000000
3         00000000010000000000
4         00000000010000000000
                  ...         
172217    01100000000000000000
172218    11000000000000000000
172219    10010000010000000000
172220    11100000000000000000
172221    01000000000000000000
Name: incStepBlockagePattern, Length: 172222, dtype: object
1         00000000010000000000
2         00000000010000000000
3         00000000010000000000
4         00000000010000000000
5         01000000000000000000
                  ...         
172217    01100000000000000000
172218    11000000000000000000
172219    10010000010000000000
172220    11100000000000000000
172221    01000000000000000000
Name: incStepBlockagePattern, Length: 172221, dtype: object


In [4]:
# print(df_RW['incStepBlockagePattern'])
lane_features = {
    "inner_shoulder": 0,
    "lane_1": 1, "lane_2": 2, "lane_3": 3, "lane_4": 4,
    "lane_5": 5, "lane_6": 6, "lane_7": 7, "lane_8": 8,
    "outer_shoulder": 9,
    "inner_slope": 10,
    "outer_slope": 11,
    "median_strip": 12,
    "weigh_station_lane": 13,
    "other": 19
}
one_hot_df=pd.DataFrame()
one_hot_df["total_block"] = df_RW['incStepBlockagePattern'].apply(lambda x: 1 if x == '1' * len(x) else 0)
one_hot_df["to_be_checked"] = df_RW['incStepBlockagePattern'].apply(lambda x: 1 if x == '2' * len(x) else 0)
# print(df_RW)
print(one_hot_df['total_block'].value_counts())
print(one_hot_df['to_be_checked'].value_counts())



for index, row in df_RW.iterrows():
    if one_hot_df.loc[index,'total_block']==1 or one_hot_df.loc[index,'to_be_checked']==1:
        column=one_hot_df.columns
        for col in column:
            if col != 'total_block' and col != 'to_be_checked':
                one_hot_df.loc[index,col] = 0
    else:
        # print(df_RW.loc[index,'incStepBlockagePattern'])
        data=df_RW.loc[index,'incStepBlockagePattern']
        for feature,idx in lane_features.items():
            if data[idx]=='1':    
                one_hot_df.loc[index,feature]=1
            else:
                one_hot_df.loc[index,feature]=0
        


df_RW=pd.concat([df_RW,one_hot_df],axis=1)
print(df_RW)

total_block
0    171496
1       725
Name: count, dtype: int64
to_be_checked
0    170737
1      1484
Name: count, dtype: int64
              incStepIncidentId incStepNum          incStepTime  \
1       20230101074552939200021          1  2023-01-01 07:45:00   
2       20230101082125894300045          1  2023-01-01 08:21:00   
3       20230101084859011100021          1  2023-01-01 08:48:00   
4       20230101084911770100021          1  2023-01-01 08:49:00   
5       20230101092618665100021          1  2023-01-01 09:26:00   
...                         ...        ...                  ...   
172217  20231031234150861100021          1  2023-10-31 23:41:00   
172218  20231031234220062400021          1  2023-10-31 23:42:00   
172219  20231031234402822100021          1  2023-10-31 23:44:00   
172220  20231031234443502100021          1  2023-10-31 23:44:00   
172221  20231031235828139400021          1  2023-10-31 23:58:00   

             incStepEndTime incStepFreewayId incStepDirection  \
1   

In [11]:
road_names = {
    10010: '國1',
    10018: '高港高架',
    10019: '國1高架',
    10020: '國2',
    10030: '國3',
    10031: '國3甲',
    10040: '國4',
    10050: '國5',
    10060: '國6',
    10080: '國8',
    10100: '國10'
}
print(df_RW)
for index, row in df_RW.iterrows():
    for id,name in road_names.items():
        if df_RW.loc[index,'incStepFreewayId']==id:
            df_RW.loc[index,name]=1
        else:
            df_RW.loc[index,name]=0

print(df_RW)


              incStepIncidentId incStepNum          incStepTime  \
1       20230101074552939200021          1  2023-01-01 07:45:00   
2       20230101082125894300045          1  2023-01-01 08:21:00   
3       20230101084859011100021          1  2023-01-01 08:48:00   
4       20230101084911770100021          1  2023-01-01 08:49:00   
5       20230101092618665100021          1  2023-01-01 09:26:00   
...                         ...        ...                  ...   
172217  20231031234150861100021          1  2023-10-31 23:41:00   
172218  20231031234220062400021          1  2023-10-31 23:42:00   
172219  20231031234402822100021          1  2023-10-31 23:44:00   
172220  20231031234443502100021          1  2023-10-31 23:44:00   
172221  20231031235828139400021          1  2023-10-31 23:58:00   

             incStepEndTime incStepFreewayId incStepDirection  \
1       2023-01-01 16:06:00            10050                1   
2       2023-01-01 08:25:00            10040                3   


In [ ]:
df_RW.to_excel('TA112.xlsx')